In [48]:
import requests
import pandas as pd 
import json
import calendar
import hvplot.pandas
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

# declare an empty list to store
# latitude and longitude of values 
# of city column
longitude = []
latitude = []
def findGeocode(state):
    try:   
        # Specify the user_agent 
        geolocator = Nominatim(user_agent="group_1")  
        return geolocator.geocode(state)  
    except GeocoderTimedOut:   
        return findGeocode(state)   

df_2019 = pd.DataFrame()
df_2020 = pd.DataFrame()
df_2021 = pd.DataFrame()

series_list = {
'LAUST010000000000003':'Alabama', 'LAUST020000000000003':'Alaska', 'LAUST040000000000003':'Arizona', 'LAUST050000000000003':'Arkansas', 'LAUST060000000000003':'California', 
'LAUST080000000000003':'Colorado', 'LAUST090000000000003':'Connecticut', 'LAUST100000000000003':'Delaware', 'LAUST120000000000003':'Florida', 'LAUST130000000000003':'Georgia', 
'LAUST150000000000003':'Hawaii', 'LAUST160000000000003':'Idaho', 'LAUST170000000000003':'Illinois', 'LAUST180000000000003':'Indiana', 
'LAUST190000000000003':'Iowa', 'LAUST200000000000003':'Kansas', 'LAUST210000000000003':'Kentucky', 'LAUST220000000000003':'Louisiana', 'LAUST230000000000003':'Maine', 
'LAUST240000000000003':'Maryland', 'LAUST250000000000003':'Massachusetts', 'LAUST260000000000003':'Michigan', 'LAUST270000000000003':'Minnesota', 'LAUST280000000000003':'Mississippi',
'LAUST290000000000003':'Missouri', 'LAUST300000000000003':'Montana', 'LAUST310000000000003':'Nebraska', 'LAUST320000000000003':'Nevada', 'LAUST330000000000003':'New Hampshire', 'LAUST340000000000003':'New Jersey', 
'LAUST350000000000003':'New Mexico', 'LAUST360000000000003':'New York', 'LAUST370000000000003':'North Carolina', 'LAUST380000000000003':'North Dakota', 'LAUST390000000000003':'Ohio', 'LAUST400000000000003':'Oklahoma', 
'LAUST410000000000003':'Oregon', 'LAUST420000000000003':'Pennsylvania', 'LAUST440000000000003':'Rhode Island', 'LAUST450000000000003':'South Carolina', 'LAUST460000000000003':'South Dakota', 'LAUST470000000000003':'Tennessee', 
'LAUST480000000000003':'Texas', 'LAUST490000000000003':'Utah', 'LAUST500000000000003':'Vermont', 'LAUST510000000000003':'Virginia', 
'LAUST530000000000003':'Washington', 'LAUST540000000000003':'West Virginia', 'LAUST550000000000003':'Wisconsin', 'LAUST560000000000003':'Wyoming',
}
df = pd.DataFrame(columns= ["State","Year","Month","Unemployment Rate"])

In [106]:
headers = {'Content-type': 'application/json'}
data1 = json.dumps({"seriesid": list(series_list.keys())[:25],"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data1, headers=headers)
json_data1 = json.loads(p.text)

data2 = json.dumps({"seriesid": list(series_list.keys())[25:],"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data2, headers=headers)
json_data2 = json.loads(p.text)

In [107]:
json_data = json_data1['Results']['series'] + json_data2['Results']['series']
json_data 

[{'seriesID': 'LAUST010000000000003',
  'data': [{'year': '2021',
    'period': 'M12',
    'periodName': 'December',
    'value': '2.6',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 2, 2022.'}]},
   {'year': '2021',
    'period': 'M11',
    'periodName': 'November',
    'value': '2.6',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 2, 2022.'}]},
   {'year': '2021',
    'period': 'M10',
    'periodName': 'October',
    'value': '2.9',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 2, 2022.'}]},
   {'year': '2021',
    'period': 'M09',
    'periodName': 'September',
    'value': '3.0',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 2, 2022.'}]},
   {'year': '2021',
    'period': 'M08',
    'periodName': 'August',
    'value': '3.7',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 2, 2022.'}]},
 

In [108]:
for series in json_data:
    state = series_list[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)
df

,State,Year,Month,Unemployment Rate
0,Montana,2021,December,2.6
1,Montana,2021,November,2.6
2,Montana,2021,October,2.6
3,Montana,2021,September,2.4
4,Montana,2021,August,3.1
...,...,...,...,...
6295,Wyoming,2019,May,3.3
6296,Wyoming,2019,April,3.4
6297,Wyoming,2019,March,4.0
6298,Wyoming,2019,February,3.9


In [109]:
df.to_pickle('pickled_unemploy_rate.pkl')

In [110]:
df = pd.read_pickle("pickled_unemploy_rate.pkl")  
df

,State,Year,Month,Unemployment Rate
0,Montana,2021,December,2.6
1,Montana,2021,November,2.6
2,Montana,2021,October,2.6
3,Montana,2021,September,2.4
4,Montana,2021,August,3.1
...,...,...,...,...
6295,Wyoming,2019,May,3.3
6296,Wyoming,2019,April,3.4
6297,Wyoming,2019,March,4.0
6298,Wyoming,2019,February,3.9


In [8]:
df_2019 = df.loc[df['Year']== '2019'].copy()
df_2019['Year Month'] = df_2019['Year'] +' '+ df_2019['Month']
df_2019 = df_2019[['Year Month', 'State','Unemployment Rate']]
df_2019.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2019 December,Alabama,2.8
2019 November,Alabama,2.7
2019 October,Alabama,2.7
2019 September,Alabama,2.8
2019 August,Alabama,3.2
...,...,...
2019 May,Wyoming,3.3
2019 April,Wyoming,3.4
2019 March,Wyoming,4.0


In [9]:
df_2020 = df.loc[df['Year']== '2020'].copy()
df_2020['Year Month'] = df_2020['Year'] +' '+ df_2020['Month']
df_2020 = df_2020[['Year Month', 'State','Unemployment Rate']]
df_2020.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2020 December,Alabama,4.1
2020 November,Alabama,4.4
2020 October,Alabama,4.9
2020 September,Alabama,6.0
2020 August,Alabama,6.7
...,...,...
2020 May,Wyoming,8.8
2020 April,Wyoming,5.3
2020 March,Wyoming,6.6


In [10]:
df_2021 = df.loc[df['Year']== '2021'].copy()
df_2021['Year Month'] = df_2021['Year'] +' '+ df_2021['Month']
df_2021 = df_2021[['Year Month', 'State','Unemployment Rate']]
df_2021.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2021 December,Alabama,2.6
2021 November,Alabama,2.6
2021 October,Alabama,2.9
2021 September,Alabama,3.0
2021 August,Alabama,3.7
...,...,...
2021 May,Wyoming,4.7
2021 April,Wyoming,5.0
2021 March,Wyoming,5.4


In [11]:
df_2019.sort_index(inplace=True,ascending=False)
df_2019 = df_2019.astype({'Unemployment Rate':'float'})
plot_2019 = df_2019.hvplot(y='Unemployment Rate', x = 'Year Month',groupby='State', rot = 90, label='Unemployment Rates-2019', ylim = (0,10))
plot_2019

:DynamicMap   [State]
   :Curve   [Year Month]   (Unemployment Rate)

In [12]:
df_2020.sort_index(inplace=True,ascending=False)
df_2020 = df_2020.astype({'Unemployment Rate':'float'})
plot_2020 = df_2020.hvplot(y='Unemployment Rate', x = 'Year Month',groupby='State', rot = 90, label='Unemployment Rates-2020', ylim = (0,10))
plot_2020

:DynamicMap   [State]
   :Curve   [Year Month]   (Unemployment Rate)

In [13]:
df_2021.sort_index(inplace=True,ascending=False)
df_2021 = df_2021.astype({'Unemployment Rate':'float'})
plot_2021 = df_2021.hvplot(y='Unemployment Rate', x = 'Year Month',groupby='State', rot = 90, label='Unemployment Rates-2021', ylim = (0,10))
plot_2021

:DynamicMap   [State]
   :Curve   [Year Month]   (Unemployment Rate)

In [14]:
plot_3 = plot_2019*plot_2020*plot_2021
plot_3

:DynamicMap   [State]
   :Overlay
      .Curve.Unemployment_Rates_hyphen_minus_2019 :Curve   [Year Month]   (Unemployment Rate)
      .Curve.Unemployment_Rates_hyphen_minus_2020 :Curve   [Year Month]   (Unemployment Rate)
      .Curve.Unemployment_Rates_hyphen_minus_2021 :Curve   [Year Month]   (Unemployment Rate)

In [15]:
longitude = []
latitude = []
states = list(series_list1.values())
states.extend(list(series_list2.values()))

In [16]:
df_state = pd.DataFrame(columns=['State'])
for i in states:
    if findGeocode(i) != None:     
        loc = findGeocode(i)
        if loc.latitude not in latitude:
            latitude.append(loc.latitude)
        if loc.longitude not in longitude:
            longitude.append(loc.longitude)
    else:
        latitude.append('')
        longitude.append('')

df_state['State'] = states
df_state['Latitude'] = latitude
df_state['Longitude'] = longitude
df_state

,State,Latitude,Longitude
0,Alabama,33.258882,-86.829534
1,Alaska,64.445961,-149.680909
2,Arizona,34.395342,-111.763275
3,Arkansas,35.204888,-92.447911
4,California,36.701463,-118.755997
5,Colorado,38.725178,-105.607716
6,Connecticut,41.650020,-72.734216
7,Delaware,38.692045,-75.401331
8,Florida,27.756767,-81.463983
9,Georgia,32.329381,-83.113737


In [17]:
s = df_2019.groupby('State').mean()
s = s.reset_index()
s

,State,Unemployment Rate
0,Alabama,3.175000
1,Alaska,5.475000
2,Arizona,4.841667
3,Arkansas,3.466667
4,California,4.108333
5,Colorado,2.600000
6,Connecticut,3.525000
7,Delaware,3.700000
8,Florida,3.233333
9,Georgia,3.600000


In [18]:
r = pd.concat([s,df_state],axis='columns',join='inner')
r = r.T.drop_duplicates().T
r['Unemployment Rate'] = r['Unemployment Rate'].astype(float)
r

,State,Unemployment Rate,Latitude,Longitude
0,Alabama,3.175000,33.258882,-86.829534
1,Alaska,5.475000,64.445961,-149.680909
2,Arizona,4.841667,34.395342,-111.763275
3,Arkansas,3.466667,35.204888,-92.447911
4,California,4.108333,36.701463,-118.755997
5,Colorado,2.600000,38.725178,-105.607716
6,Connecticut,3.525000,41.65002,-72.734216
7,Delaware,3.700000,38.692045,-75.401331
8,Florida,3.233333,27.756767,-81.463983
9,Georgia,3.600000,32.329381,-83.113737


In [19]:
r.hvplot.points(
    x='Longitude', 
    y='Latitude',
    xlabel='Longitude',
    ylabel='Latitude',
    geo=True, 
    tiles='OSM', 
    frame_width=700, 
    frame_height=500,
    label='2019 Unemployment',
    hover_cols=['State','Unemployment Rate'],
    size='Unemployment Rate',
    color= 'Unemployment Rate',
    scale=5
)

:Overlay
   .Tiles.I                    :Tiles   [x,y]
   .Points.A_2019_Unemployment :Points   [Longitude,Latitude]   (Unemployment Rate,State)